In [ ]:
!pip install googletrans==3.1.0a0
! pip install langchain_community
!pip install pdfplumber
!pip install --upgrade cryptography pyOpenSSL

# Funções

In [13]:
from googletrans import Translator

def translate_text(text):
    translator = Translator()
    translation = translator.translate(text, dest='en')
    return translation.text

In [14]:
from langchain_community.document_loaders import PDFPlumberLoader

def ler_pdf(file_path, view=False):
    # Crie um objeto PDFPlumberLoader com o caminho do arquivo
    loader = PDFPlumberLoader(file_path)

    # Carregue o conteúdo do PDF
    documents = loader.load()

    if view:
        # Exiba o conteúdo de cada página do documento
        for idx, document in enumerate(documents, start=1):
            print(f"Conteúdo da página {idx}:")
            print(document.page_content)
            print("-" * 50)
    
    return documents

In [15]:
import re

def limpar_documento(documents, view = False):
    doc = []
    # Exiba o conteúdo de cada página do documento
    for idx, document in enumerate(documents, start=1):
        if view:
            print(f"Conteúdo da página {idx}:")
        texto_completo = document.page_content

        # Remover quebras de linha desnecessárias e substituir por espaços
        texto_completo = re.sub(r'\n+', ' ', texto_completo)
        texto_completo = ' '.join(texto_completo.split())
        texto_completo = translate_text(texto_completo)

        # Separar o texto em linhas
        linhas = texto_completo.split('--------------------------------------------------')
        doc += linhas
        if view:
            # Limpar cada linha e exibir
            for linha in linhas:
                linha_limpa = re.sub(r'\s+', ' ', linha.strip())
                if linha_limpa:
                    print(linha_limpa)
                    print('-' * 50)
    
    return doc

In [16]:
from transformers import pipeline, AutoTokenizer
import torch

modelo_id = "facebook/bart-large-cnn"

# Check if GPU is available
device = -1 #0 if torch.cuda.is_available() else -1
tokenizer = AutoTokenizer.from_pretrained(modelo_id, max_new_tokens=100, device=device) #truncation=True
summarizer = pipeline("summarization", model=modelo_id, tokenizer=tokenizer, device=device)

In [17]:
def sumarizar_documento(doc, view=False):
    resumos = []
    for i in range(len(doc)):
        if view:
            print(f"sumarizando pagina: ({i+1}/{len(doc)})")
        try:
            if len(doc[i]) > 0:
                doc[i] = ' '.join(doc[i].split())
                num_token = len(tokenizer.tokenize(doc[i]))
                resumo = summarizer(doc[i],
                                    max_length=num_token,
                                    min_length=int(num_token*0.6),
                                    do_sample=True,
                                    num_beams=6,
                                    top_k=50,
                                    temperature=0.1)
                resumos += resumo
        except Exception as e:
            print(f"Erro ao processar doc[{i}]: {e}")
            continue  # Ir para o próximo doc[i]
        
    if view:
        for resumo in resumos:
            print("\n")
            print(resumo["summary_text"])
            print("\n\n")
    return resumos

In [18]:
import re


def get_name(file_name):
  return file_name[:len(file_name)-4] + ".txt"


def savar_resumo_txt(resumos, output_file):
    # Nome do arquivo de saída  
    output_file = get_name(output_file)
    # Abra o arquivo de saída em modo de escrita
    with open(output_file, "w", encoding="utf-8") as file:
        # Exiba o conteúdo de cada página do documento
        for i in range(len(resumos)):
            #file.write(f"Conteúdo DDda página: ") # TENTA REMOVER ISSO PRA VER SE DA CERTO
            texto_completo = resumos[i]['summary_text']

            # Remover quebras de linha desnecessárias e substituir por espaços
            texto_completo = re.sub(r'\n+', ' ', texto_completo)
            # Separar o texto em linhas
            linhas = texto_completo.split('--------------------------------------------------')
            # Limpar cada linha e escrever no arquivo
            for linha in linhas:
                linha_limpa = re.sub(r'\s+', ' ', linha.strip())
                if linha_limpa:
                    file.write(linha_limpa + "\n")
                    #file.write('-' * 50 + "\n")

# Gerar txt

In [19]:
from tqdm import tqdm
import os 

local_pdf = "PDF"
local_txt = "TXT"

lista_pdf = os.listdir(local_pdf)

for i in tqdm(range(len(lista_pdf)), desc="Processando PDFs"):
    pdf = lista_pdf[i]
    file_path = os.path.join(local_pdf, pdf)
    output_file = os.path.join(local_txt, pdf)
    
    documents = ler_pdf(file_path)
    doc = limpar_documento(documents)
    resumos = sumarizar_documento(doc)
    savar_resumo_txt(resumos, output_file)

Processando PDFs:  71%|███████▏  | 92/129 [2:18:30<53:28, 86.70s/it]   

Erro ao processar doc[2]: Input length of decoder_input_ids is 1, but `max_length` is set to 1. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.


Processando PDFs: 100%|██████████| 129/129 [3:12:56<00:00, 89.74s/it]  
